In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
sklearn.set_config(transform_output='pandas')

pd.options.display.max_columns = 40
pd.set_option('display.float_format', lambda x: '%.3f' % x)


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from category_encoders import CatBoostEncoder, TargetEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostRegressor


import joblib

In [4]:
df = pd.read_csv('data/car_data.csv')

In [5]:
df.head(2)

,url,time,price,brand,model,поколение,тип продавца,продавец,год выпуска,пробег,птс,состояние,модификация,объем двигателя,тип двигателя,коробка передач,привод,комплектация,тип кузова,цвет,руль,доп опции,авито оценка,автотека,расположение,описание,фотографии авто
0,https://www.avito.ru/moskva/avtomobili/toyota_...,2023-08-17 07:28:31 +0300,1700000,Toyota,Land Cruiser,200 (2007—2012),частник,олег,2010,110000,NaN,требует ремонта,4.5 D AT (235 л.с.),4.500,дизель,автомат,полный,Luxe 60Th,внедорожник,белый,левый,"Фары: Противотуманные, Омыватели фар, Адаптивн...",-,"2 владельца по ПТС, Проверка на ДТП: тяжесть п...",Москва,"Продаю свой авто , юридически чистая, после ДТ...",https://00.img.avito.st/image/1/1.6jfbh7a5Rt7t...
1,https://www.avito.ru/moskva/avtomobili/skoda_k...,2023-01-07 15:50:46 +0300,3500000,Skoda,Kodiaq,I (2016—2023),частник,Пользователь,2019,71000,NaN,отличное,2.0 TDI 4WD DSG (150 л.с.),2.000,дизель,робот,полный,Style,внедорожник,коричневый,левый,"Фары: Светодиодные, Противотуманные, Омыватели...",2542300 - 2866899,"1 владелец по ПТС, ДТП не найдены, Реальный пр...",Москва,ПРОДАЖА ТОЛЬКО ЧАСТНОМУ ЛИЦУ! ВСЕХ ДИЛЕРОВ БЛО...,https://17.img.avito.st/image/1/1.mHlOubaxNJB4...


In [6]:
df.columns

Index(['url', 'time', 'price', 'brand', 'model', 'поколение', 'тип продавца',
       'продавец', 'год выпуска', 'пробег', 'птс', 'состояние', 'модификация',
       'объем двигателя', 'тип двигателя', 'коробка передач', 'привод',
       'комплектация', 'тип кузова', 'цвет', 'руль', 'доп опции',
       'авито оценка', 'автотека', 'расположение', 'описание',
       'фотографии авто'],
      dtype='object')

In [7]:
dropped_columns = ['url', 'time', 'тип продавца', 'продавец', 'птс', 'доп опции', 'авито оценка', 'автотека', 'описание', 'фотографии авто']

target_encodig_columns = ['brand', 'model', 'поколение', 'состояние', 'модификация', 'тип двигателя', 
                          'коробка передач', 'привод', 'комплектация', 'тип кузова', 'цвет', 'руль', 'расположение']

target_column = ['price']

num_columns = ['год выпуска', 'пробег', 'объем двигателя']

In [8]:
len(dropped_columns) + len(target_encodig_columns) + len(target_column) + len(num_columns)

27

In [9]:
df.shape[1]

27

In [10]:
encoder = ColumnTransformer(
    transformers = [
        ('column_dropper', 'drop', dropped_columns),
        ('target_encoder', CatBoostEncoder(), target_encodig_columns),
    ],
    remainder='passthrough',
    verbose_feature_names_out = False
)

scaler = StandardScaler()

preprocessor = Pipeline(
    steps = [
        ('encoder', encoder),
        ('sacler', scaler)
    ]
)

In [11]:
preprocessor

Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('column_dropper', 'drop',
                                                  ['url', 'time',
                                                   'тип продавца', 'продавец',
                                                   'птс', 'доп опции',
                                                   'авито оценка', 'автотека',
                                                   'описание',
                                                   'фотографии авто']),
                                                 ('target_encoder',
                                                  CatBoostEncoder(),
                                                  ['brand', 'model',
                                                   'поколение', 'состояние',
                                                   'модификация',
                                                   'тип двигателя',
                                                   'коробка передач', 'привод',
                                                   'комплектация', 'тип кузова',
                                                   'цвет', 'руль',
                                                   'расположение'])],
                                   verbose_feature_names_out=False)),
                ('sacler', StandardScaler())])

In [12]:
X, y = df.drop('price', axis=1), df['price']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
ml_pipeline = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', CatBoostRegressor(random_state=42)),
    ]
)



In [14]:
ml_pipeline.fit(X_train, y_train, model__verbose=False)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('encoder',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('column_dropper',
                                                                   'drop',
                                                                   ['url',
                                                                    'time',
                                                                    'тип '
                                                                    'продавца',
                                                                    'продавец',
                                                                    'птс',
                                                                    'доп опции',
                                                                    'авито '
                                                                    'оценка',
                                                                    'автотека',
                                                                    'описание',
                                                                    'фотографии '
                                                                    'авто']),
                                                                  ('target_encoder',
                                                                   CatBoostEncoder(),
                                                                   ['brand',
                                                                    'model',
                                                                    'поколение',
                                                                    'состояние',
                                                                    'модификация',
                                                                    'тип '
                                                                    'двигателя',
                                                                    'коробка '
                                                                    'передач',
                                                                    'привод',
                                                                    'комплектация',
                                                                    'тип '
                                                                    'кузова',
                                                                    'цвет',
                                                                    'руль',
                                                                    'расположение'])],
                                                    verbose_feature_names_out=False)),
                                 ('sacler', StandardScaler())])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x174accf40>)])

In [15]:
ml_pipeline.score(X_valid, y_valid)

0.8855663434697706

In [16]:
model = ml_pipeline['model']

In [17]:
pd.DataFrame(index=model.feature_names_, data=model.feature_importances_, columns=['importance']).sort_values(by='importance', ascending=False)

,importance
модификация,21.493
поколение,18.405
объем двигателя,17.615
пробег,11.783
год выпуска,7.963
model,6.839
brand,5.577
тип кузова,2.689
привод,2.251
коробка передач,1.377


In [18]:
joblib.dump(ml_pipeline, 'model/ml_pipeline.pkl')

['model/ml_pipeline.pkl']

In [19]:
ml_pipeline

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('encoder',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('column_dropper',
                                                                   'drop',
                                                                   ['url',
                                                                    'time',
                                                                    'тип '
                                                                    'продавца',
                                                                    'продавец',
                                                                    'птс',
                                                                    'доп опции',
                                                                    'авито '
                                                                    'оценка',
                                                                    'автотека',
                                                                    'описание',
                                                                    'фотографии '
                                                                    'авто']),
                                                                  ('target_encoder',
                                                                   CatBoostEncoder(),
                                                                   ['brand',
                                                                    'model',
                                                                    'поколение',
                                                                    'состояние',
                                                                    'модификация',
                                                                    'тип '
                                                                    'двигателя',
                                                                    'коробка '
                                                                    'передач',
                                                                    'привод',
                                                                    'комплектация',
                                                                    'тип '
                                                                    'кузова',
                                                                    'цвет',
                                                                    'руль',
                                                                    'расположение'])],
                                                    verbose_feature_names_out=False)),
                                 ('sacler', StandardScaler())])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x174accf40>)])